In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!conda install beautifulsoup4 --yes
from bs4 import BeautifulSoup
!conda install requests --yes
import requests

%matplotlib inline

Fetching package metadata .......
Solving package specifications: ..........

# All requested packages already installed.
# packages in environment at /Users/veenakumar/anaconda2:
#
beautifulsoup4            4.5.1                    py27_0  
Fetching package metadata .......
Solving package specifications: ..........

# All requested packages already installed.
# packages in environment at /Users/veenakumar/anaconda2:
#
requests                  2.11.1                   py27_0  


In [ ]:
#***NOTE*** Commented this cell out because it only needed to be run ONCE.


# #STEP 1: This is the first "for loop". Scrape Box Office Mojo to get the html data. For this project, I am using the top 100 
# #domestic gross movies from each year starting from 2000.
# #STEP 2: This is the second "for loop". Define a function to grab only the urls from each of the above 16 pages. Theoretically
# #there should be 1600 urls. 

# links = []

# for yr in range(2000,2017):
#     url = 'http://www.boxofficemojo.com/yearly/chart/?yr={}&view=releasedate&view2=domestic&sort=gross&order=DESC&&p=.htm'.format(yr)
#     response = requests.get(url)
#     page = response.text
#     soup = BeautifulSoup(page)
#     for link in soup.findAll('a'):
#         if 'href' in link.attrs:    
#             if 'movies/?id' in link['href']:
#                 links.append('http://boxofficemojo.com{0}'.format(link['href']))    

# #Gives me a total of 1717 links.

In [ ]:
#***NOTE*** Commented this cell out because it only needed to be run ONCE.


# #STEP 3: Grabbing the html data for individual movie pages and saving it to my local drive. 
# #Advantage of having it local means I will not have to keep accessing Box Office Mojo over
# #multiple days every time I want to retrieve data.

# def grab_and_save_pages(links):
#     for i in links:
#         response = requests.get(i)
#         page = response.text
#         soup = BeautifulSoup(page)
#         soup = soup.prettify("utf-8")
#         title = i.split('=')[1]
#         with open('/Users/veenakumar/Desktop/Projects/Luther/BomData/%s'%title, "wb") as f:
#             f.write(soup)
            
# grab_and_save_pages(links)

# #Resulted in 1701 files

In [3]:
#Running the above step again to get the Foreign data page. 
import os
path = '/Users/veenakumar/Desktop/Projects/Luther/BomData/'

for filename in os.listdir(path):
    if filename.endswith('htm'):
        url = 'http://www.boxofficemojo.com/movies/?page=intl&id={}'.format(filename)
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page)
        soup = soup.prettify("utf-8")     
        title = url.split('=')[2]
        with open('/Users/veenakumar/Desktop/Projects/Luther/BomForeign/%s'%title, "w") as f:
            f.write(soup)
    

In [ ]:
adjusted = {2000: 5.39, 2001: 5.66, 2002: 5.81, 2003: 6.03, 2004: 6.21, 2005: 6.41, 2006: 6.55, 2007: 6.88, 2008: 7.18, 2009: 7.50, 2010: 7.89, 2011: 7.93, 2012: 7.96, 2013: 8.13, 2014: 8.17, 2015: 8.43, 2016: 8.66}

import re

In [ ]:
def get_values_tags(datafile, field_name):
    soup = BeautifulSoup(open(datafile))
    obj = soup.find(field_name).text
    return obj

def get_values_text(datafile, field_name):
    soup = BeautifulSoup(open(datafile))
    obj = soup.find(text=re.compile(field_name))
    if not obj: 
        return None
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text 
    else:
        return None
    

In [ ]:
import os

path = '/Users/veenakumar/Desktop/Projects/Luther/BomData/'

fields_tags = ['title']
fields_text_neighbor = ['Domestic Total','Distributor','Release Date','Genre:','Runtime','MPAA Rating','Production Budget']
movie_dict = {}

def make_dict_tags(fields):
    for field in fields:
        for filename in os.listdir(path):
            if filename.endswith('htm'): 
                datafile = path+filename
                if field in movie_dict:
                    movie_dict[field].append(get_values_tags(datafile,field))
                else:
                    movie_dict[field] = [get_values_tags(datafile,field)]

                    
def make_dict_text(fields):
    for field in fields:
        for filename in os.listdir(path):
            if filename.endswith('htm'): 
                datafile = path+filename
                if field in movie_dict:
                    movie_dict[field].append(get_values_text(datafile,field))
                else:
                    movie_dict[field] = [get_values_text(datafile,field)]

In [ ]:
#Getting all the titles.
make_dict_tags(fields_tags)

In [ ]:
#Getting a few more fields.
make_dict_text(fields_text_neighbor)

In [ ]:
#Turn dictionary so far into a df.
movie_data = pd.DataFrame(movie_dict)

In [ ]:
#Stripping the spaces around the values
for value in movie_dict.itervalues():
    for i in range(len(value)):
        value[i] = str(value[i])
        value[i] = value[i].strip()

In [ ]:
#**NOTE: Only run this once. Specifically for titles. Commented out so it won't be run twice.
# #Stripping titles.
# for value in movie_dict.itervalues():
#     for i in range(len(value)):
#         value[i] = value[i].split('(')[0].strip()

In [ ]:
#Saving the data so far!
movie_data.to_csv('/Users/veenakumar/Desktop/Projects/Luther/Luther_Movies_Data.csv')

In [ ]:
#Parse Revenue

def foreign(path,texts):
    for filename in os.listdir(path):
        if filename.endswith('htm'):
            datafile = path+filename
            soup = BeautifulSoup(open(datafile))                
            htmltag = soup.find("div", class_="mp_box_content").find(text=re.compile(texts))
            if htmltag:
                field_value = htmltag.parent.parent.findNextSibling().text.strip()
            else:
                field_value = None
            if texts in movie_dict:
                    movie_dict[texts].append(field_value)
            else:
                movie_dict[texts] = [field_value]



In [ ]:
movie_data.head()

In [ ]:
#Extracting release year.
movie_data['Release Yr'] = [row[-4:] for row in movie_data['Release Date']]

In [ ]:
#Adjust for inflation.

current = 8.66
adjusted = {2000: 5.39, 2001: 5.66, 2002: 5.81, 2003: 6.03, 2004: 6.21, 2005: 6.41, 2006: 6.55, 2007: 6.88, 2008: 7.18, 2009: 7.50, 2010: 7.89, 2011: 7.93, 2012: 7.96, 2013: 8.13, 2014: 8.17, 2015: 8.43, 2016: 8.66}

def convert_to_num(dataframe,column):
    for row in dataframe[column]:
        num = row.replace('$','').replace(',','')
        if num != 'None':
            num = int(num)
        else:
            num = 0
        return num




# for i in range(len(movie_data)):
#     row = movie_data['Release Yr'][i]
#     rate = current/adjusted[int(row)]
#     movie_data['Domestic Total'][i] = int(movie_data['Domestic Total'][i])*rate
# return movie_data['Domestic Total'][i]


In [ ]:
convert_to_num(movie_data,'Domestic Total')